# Incremental Ernollment Trial with voxceleb1
---------

trial 길이가 충분히 긴 set을 만들고자 한다.

## Environment

In [2]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pickle
import pandas as pd
from utils import key2df

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [8]:
from batch_sv_system import get_embeds, cosine_sim, compute_plda_score
from utils import key2df, df2dict, get_id2idx

## Dataframes & embeddings

In [4]:
# voxc1_df['label'] = voxc1_df.groupby('spk').ngroup().values
# voxc1_df = voxc1_df.sort_values('id').reset_index(drop=True)
# voxc1_df = voxc1_df.fillna('test')
# voxc1_meta = pd.read_pickle("/dataset/SV_sets/voxceleb1/dataframes/voxc1_meta.pkl")
# spk2gender = voxc1_meta.Gender.to_dict()
# voxc1_df['gender'] = voxc1_df.spk.apply(lambda x: spk2gender[x])
# voxc1_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1.csv", index=False)

# voxc1_dev_df = voxc1_df[voxc1_df.spk.isin(dev_spks)]
# voxc1_eval_df = voxc1_df[voxc1_df.spk.isin(eval_spks)]
# voxc1_dev_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_dev.csv", index=False)
# voxc1_eval_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_eval.csv", index=False)

In [9]:
voxc1_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1.csv")
spk_uttr_stat = voxc1_df.spk.value_counts()
voxc1_meta = pd.read_pickle("/dataset/SV_sets/voxceleb1/dataframes/voxc1_meta.pkl")
spk2gender = voxc1_meta.Gender.to_dict()

In [21]:
from utils import key2df

embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
si_embeds = np.load(embed_dir+"ln_lda_si_embeds.npy")
sv_embeds = np.load(embed_dir+"ln_lda_sv_embeds.npy")
si_keys = pickle.load(open(embed_dir + "/si_keys.pkl", "rb"))
sv_keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)
si_df = key2df(si_keys)

In [63]:
from ioffe_plda.verifier import Verifier
py_plda_model = Verifier()
py_plda_model = Verifier(pickle.load(open("py_plda_model_ln_lda.pkl", "rb")))

## Hold-out Developement set

utterance가 150개 넘는 것들을 evaluation spks로 빼고 나머지는 developement spk로 빼었다.  
그리고 developement spk 음성을 이용해서 threshold를 정하기 위한 trial을 만든다.

In [66]:
dev_spks = spk_uttr_stat[(spk_uttr_stat < 150)].index.tolist()
dev_uttrs = voxc1_df[voxc1_df.spk.isin(dev_spks)][['id', 'spk', 'gender', 'session']]
eval_spks = spk_uttr_stat[spk_uttr_stat >= 150].index.tolist()
eval_uttrs = voxc1_df[voxc1_df.spk.isin(eval_spks)][['id', 'spk', 'gender', 'session']]

In [64]:
print(f"number of dev spks: {len(dev_spks)}")
print(f"number of eval spks: {len(eval_spks)}")

number of dev spks: 940
number of eval spks: 311


In [7]:
dev_uttrs.to_pickle("trials/dev940_eval311/dev_uttrs.pkl")

In [7]:
Dev_uttrs = pd.read_pickle("trials/dev940_eval311/dev_uttrs.pkl")
dev_cohort_embeds = get_embeds(dev_uttrs.sample(n=5000).id, plda_sv_embeds, plda_id2idx, norm=False)
np.save("trials/dev940_eval311/dev_cohort_embeds.npy", dev_cohort_embeds)

dev_trials = pd.read_pickle("trials/enr306/trial_for_dev.pkl")

## Cohort Set

In [58]:
cohort_ids = dev_uttrs.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=20)).id
# cohort_embeds = get_embeds(cohort_ids, plda_sv_embeds, plda_id2idx, norm=True)
# np.save("trials/dev317_eval934/cohort_embeds.npy", cohort_embeds)

In [62]:
np.save("trials/dev940_eval311/dev_cohort_ids.npy", np.array(cohort_ids))

## Random trials

###  Dev trials

In [66]:
dev_trials = []

for dev_spk in dev_spks: # TODO: multiple enrolled speakers case
    target_gender = spk2gender[dev_spk]
    target_uttrs = dev_uttrs[dev_uttrs.spk == dev_spk]
    nonTarget_uttrs = dev_uttrs[(dev_uttrs.spk != dev_spk)]
    nonTarget_uttrs = nonTarget_uttrs[nonTarget_uttrs.gender == target_gender]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=20)
    test_target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
    test_nonTarget_uttrs = nonTarget_uttrs.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=10))
    test_trial = pd.concat([test_target_uttrs, test_nonTarget_uttrs])
    dev_trials += [[dev_spk, np.array(enr_uttrs.id), 
               (np.array(test_trial.id), np.array(test_trial.label)),
             ]]
    
save_dir = "./trials/dev317_eval934/dev_random_enr20_spk10_gender/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(dev_trials, open(save_dir + "/trials.pkl", "wb"))    

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


###  Eval trials

In [82]:
eval_target_pool = eval_uttrs.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=60))
eval_nonTarget_pool = eval_uttrs.drop(index=eval_target_pool.index)

#### normal 

In [64]:
eval_trials = []

for eval_spk in eval_spks: # TODO: multiple enrolled speakers case
    target_gender = spk2gender[eval_spk]
    target_uttrs = eval_target_pool[eval_target_pool.spk == eval_spk]
    nonTarget_uttrs = eval_nonTarget_pool[(eval_nonTarget_pool.spk != eval_spk)]
    nonTarget_uttrs = nonTarget_uttrs[nonTarget_uttrs.gender == target_gender]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=10)
    test_target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
#     test_nonTarget_uttrs = nonTarget_uttrs.sample(n=4950)
    test_nonTarget_uttrs = nonTarget_uttrs.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=10))
    test_trial = pd.concat([target_uttrs, test_nonTarget_uttrs])
    eval_trials += [(eval_spk, np.array(enr_uttrs.id), test_trial)]
    
save_dir = "./trials/dev317_eval934/eval_random_enr10_tar50_spk10_gender/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(eval_trials, open(save_dir + "/trials.pkl", "wb"))    

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Hard trials

### Eval trial 

In [108]:
# eval_nonTarget_pool = eval_uttrs.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=50))
# eval_target_pool = eval_uttrs.drop(index=eval_nonTarget_pool.index)

In [ ]:
eval_trials = []

for eval_spk in eval_spks: # TODO: multiple enrolled speakers case
    print(eval_spk)
    target_gender = spk2gender[eval_spk]
    target_uttrs = eval_uttrs[eval_uttrs.spk == eval_spk]
#     nonTarget_uttrs = eval_nonTarget_pool[(eval_nonTarget_pool.spk != eval_spk)]
    nonTarget_uttrs = dev_uttrs
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=3)
    test_target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
    
    enr_embeds = get_embeds(enr_uttrs.id, sv_embeds, id2idx)
    nonTarget_embeds = get_embeds(nonTarget_uttrs.id, sv_embeds, id2idx)
    scores = py_plda_model.score_avg(enr_embeds, nonTarget_embeds)
    
    hard_neg_trials = np.argsort(scores.mean(0), axis=0)[-len(test_target_uttrs)*9:].ravel()
    test_nonTarget_uttrs = nonTarget_uttrs.iloc[hard_neg_trials]
    print(test_nonTarget_uttrs.spk.value_counts()[:5])
    test_trial = pd.concat([target_uttrs, test_nonTarget_uttrs])
    eval_trials += [(eval_spk, np.array(enr_uttrs.id), test_trial)]
    
save_dir = "./trials/eval165/eval_hard_enr10_tarAll_devNonTarget/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(eval_trials, open(save_dir + "/trials.pkl", "wb"))    

## Dev Target trials 

In [38]:
import itertools

dev_combs = dev_uttrs.groupby("spk", group_keys=False).apply(lambda x: itertools.combinations(x.id, 2))
dev_target_trials = []
for comb in dev_combs:
    dev_target_trials += list(comb)

In [44]:
dev_target_trial_df = pd.DataFrame(dev_target_trials, columns=['enr', 'tst'])

In [55]:
dev_target_trial_set = dev_target_trial_df.groupby('enr', group_keys=False).apply(lambda x: list(x.tst)).to_dict()

In [56]:
len(dev_target_trial_set)

82141

In [57]:
pickle.dump(dev_target_trial_set, open("trials/dev940_eval311/dev_target_trials.pkl", "wb"))